In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline

# import Sastrawi
# import nltk

import re
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import cross_validate
# from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
# from nltk.tag import CRFTagger
# from collections import Counter
# import warnings
# warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', 300)
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def convert_lexicon_score_to_word(score):
    if score > 0:
        return " ".join(['selamat'] * score)
    else:
        return " ".join(['bodoh'] * -score)

In [3]:
def convert_emoticon_score_to_word(score):
    if score > 0:
        return " ".join(['selamat'] * score)
    else:
        return " ".join(['bodoh'] * -score)

In [4]:
train_data = pd.read_csv('tdp.csv', encoding='Latin')
tambahan_data = pd.read_csv('tambahan_preprocess.csv', encoding='Latin')[0:120]
train_data = pd.concat([train_data, tambahan_data], ignore_index=True)
train_data['tweet'].fillna('', inplace=True)
tester_data = pd.read_csv('tdp_test.csv', encoding='Latin')
tester_data['tweet'].fillna('', inplace=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, use_idf=True)
train_data_tfidf = tfidf.fit_transform(train_data['tweet'])
tester_data_tfidf = tfidf.transform(tester_data['tweet'])

train_data['temp'] = train_data['lexicon_score'].apply(convert_lexicon_score_to_word)
train_data['temp2'] = train_data['emoticon_score'].apply(convert_emoticon_score_to_word)
train_data['tweet'] = train_data['tweet'] + ' ' + train_data['temp'] + ' ' + train_data['temp2']

tester_data['temp'] = tester_data['lexicon_score'].apply(convert_lexicon_score_to_word)
tester_data['temp2'] = tester_data['lexicon_score'].apply(convert_emoticon_score_to_word)

tester_data['tweet'] = tester_data['tweet'] + ' ' + tester_data['temp'] + ' ' + tester_data['temp2']

train_data_tfidf_df = pd.DataFrame(train_data_tfidf.toarray(), columns=tfidf.get_feature_names())
tester_data_tfidf_df = pd.DataFrame(tester_data_tfidf.toarray(), columns=tfidf.get_feature_names())

In [ ]:
train_data_tfidf_df['lexicon_score'] = train_data['lexicon_score']
train_data_tfidf_df['emoticon_score'] = train_data['emoticon_score']
train_data_tfidf_df['jj'] = train_data['jj']
train_data_tfidf_df['neg'] = train_data['neg']
train_data_tfidf_df['exclamation'] = train_data['exclamation']
train_data_tfidf_df['ortografi'] = train_data['ortografi']
train_data_tfidf_df['lexicon_pos_score'] = train_data['lexicon_pos_score']
train_data_tfidf_df['lexicon_neg_score'] = train_data['lexicon_neg_score']


In [ ]:
tester_data_tfidf_df['lexicon_score'] = tester_data['lexicon_score']
tester_data_tfidf_df['emoticon_score'] = tester_data['emoticon_score']
tester_data_tfidf_df['jj'] = tester_data['jj']
tester_data_tfidf_df['neg'] = tester_data['neg']
tester_data_tfidf_df['exclamation'] = tester_data['exclamation']
tester_data_tfidf_df['ortografi'] = tester_data['ortografi']
tester_data_tfidf_df['lexicon_pos_score'] = tester_data['lexicon_pos_score']
tester_data_tfidf_df['lexicon_neg_score'] = tester_data['lexicon_neg_score']




In [ ]:
from sklearn.decomposition import TruncatedSVD
t_svd = TruncatedSVD()
feature_names = tfidf.get_feature_names()
vocab = tfidf.vocabulary_

In [ ]:
t_svd.fit(train_data_tfidf)
best_features = [feature_names[i] for i in t_svd.components_[0].argsort()[::-1]]

In [ ]:
features = best_features[0:1740] + ['lexicon_score', 'emoticon_score', 'neg', 'jj']
target = 'sentimen'

X, y = train_data_tfidf_df[features], train_data[target].values
Xx = tester_data_tfidf_df[best_features]
Xx.to_csv('Xx.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVC, LinearSVC, SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=46)
classifier = BernoulliNB()
# classifier = VotingClassifier(estimators=[('bnb', BernoulliNB()), ('mnlb', MultinomialNB())], 
#                               voting='soft')

classifier.fit(X_train, y_train)
print(accuracy_score(classifier.predict(X_test), y_test))

classifier.fit(X, y)
print(accuracy_score(classifier.predict(X_test), y_test))

In [ ]:
# for j in range(800, 2000, 10):
#     features = best_features[0:j] + ['emoticon_score', 'lexicon_score', 'jj', 'neg']
#     target = 'sentimen'

#     X, y = train_data_tfidf_df[features], train_data[target].values
#     print('------------------' + str(j) + '--------------')
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=46)
#     classifier = BernoulliNB()

#     classifier.fit(X_train, y_train)
#     print(accuracy_score(classifier.predict(X_test), y_test))

#     classifier.fit(X, y)
#     print(accuracy_score(classifier.predict(X_test), y_test))
    

In [ ]:
predicted = classifier.predict(Xx)

In [ ]:
tester_data['predicted'] = predicted
tester_data.to_csv('result20.csv', index=False, header=False, columns=['tweetID', 'predicted'])

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
# from sklearn.svm import LinearSVC, SVC, NuSVC
# from sklearn.naive_bayes import BernoulliNB, MultinomialNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


# classifiers = [
#     ('Decission Tree', DecisionTreeClassifier()),
#     ('Logistic Regression', LogisticRegression()),
#     ('SVM', LinearSVC()),
#     ('Multinomial Naive Bayes', MultinomialNB()),
#     ('KNN', KNeighborsClassifier()),
#     ('Ensemble', GradientBoostingClassifier())
# ]

# from sklearn.model_selection import KFold

# train_scores = 0
# test_scores = 0

# kfold = KFold(n_splits=10, random_state=46)
# dt = MultinomialNB()
# for train_index, test_index in kfold.split(X):
#     X_train, y_train = X[train_index], y[train_index]
#     X_test, y_test = X[test_index], y[test_index]
#     dt.fit(X_train, y_train)
#     train_scores += accuracy_score(dt.predict(X_train), y_train)
#     test_scores += accuracy_score(dt.predict(X_test), y_test)
    
# print(train_scores / 10)
# print(test_scores / 10)

In [ ]:
# predicted = dt.predict(Xx)

In [ ]:
# tester_data['predicted'] = predicted

In [ ]:
# tester_data.to_csv('results14.csv', index=False, header=False, columns=['tweetID', 'predicted'])

In [ ]:
import pickle
import pandas as pd
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
classifier = pickle.load(open('model.sav', 'rb'))
Xx = pd.read_csv('Xx.csv')

In [ ]:
predicted = classifier.predict(Xx)